# 第 7 章 用 RAG 讓模型增加額外知識

In [ ]:
!pip install langchain langchain_openai rich

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.4/116.4 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.9/312.9 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.1 MB/s eta 0:00:00
  Attempting uninstall: packag

In [ ]:
# 匯入套件和金鑰
from google.colab import userdata
from rich import print as pprint
import os
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
from langchain_openai import ChatOpenAI
chat_model = ChatOpenAI()

## 7-1 什麼是 RAG?

### RAG 第一步：資料匯入

In [ ]:
!pip install pypdf chromadb rapidocr-onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 47.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 59.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 8.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (py

extract_images 設定 True 可以解析帶有圖片的 PDF

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader(file_path='https://ppt.cc/f9nc5x')

In [ ]:
docs = loader.load()
pprint(docs[0])

Document(
    page_content='汽車法規是非題   \n第1頁/共36頁                        分類編號欄位說明   \n分類編\n號  
分類項目內容   \n01  路口安全 (有號誌路口、無號誌路口、停讓行人 )  \n02  轉彎(左右轉、迴轉 ) \n03  行駛中應注意事項
(保持安全車距、注意前車狀況 ) \n04  正確使用燈光 (頭燈、霧燈、方向燈 ) \n05  注意大型車行駛及轉彎 
(內輪差、視野死角、不並行 ) \n06  貨物裝載 (防止掉落或滲漏 ) \n07 事故預防及處理 (預防國道二次事故、急救常識 ) \n08
禁止不當行為 (酒駕、不使用手機、危險駕駛 ) \n09 行車檢查 (設備、燈光 ) \n10 
其他(平交道、強制險、環保駕駛、特殊天候、駕駛道德 ) \n  ',
    metadata={'source': 'https://ppt.cc/f9nc5x', 'page': 0}
)

In [ ]:
from langchain.indexes import VectorstoreIndexCreator
from langchain_openai import OpenAIEmbeddings
embeddings_model=OpenAIEmbeddings(model='text-embedding-3-large')
index = VectorstoreIndexCreator(
    embedding=embeddings_model).from_loaders([loader])

In [ ]:
query = "酒後開車且酒精濃度超過規定標準應罰款多少?"
response = index.query(llm=chat_model, question=query)
print(response)

如果酒後開車且酒精濃度超過規定標準，根據提供的汽車法規資訊，應處以新臺幣30,000至120,000元的罰鍰。


### RAG 第二步：資料分割

In [ ]:
test_doc = docs[1].page_content[:200]
test_doc

'汽車法規是非題   \n第2頁/共36頁    \n題號 答案 題   目  分類\n編號  \n001  ○  尊重生命是駕駛道德最重要的一點，我們開車時要處處顧 及行人，尤\n其應該注意讓老弱婦孺身心障礙者優先通行。   10 \n002  X  遵守交通法規與秩序，只算是優良駕駛人，與駕駛道德無關。   10 \n003  ○  汽油著火時，應用滅火器、泥沙或用水浸濕棉被、衣服覆蓋撲滅。   07 \n00'

In [ ]:
from langchain_text_splitters import (
    CharacterTextSplitter, RecursiveCharacterTextSplitter)

In [ ]:
text_splitter = CharacterTextSplitter(separator='',
                                      chunk_size=10,
                                      chunk_overlap=2)
chunks = text_splitter.split_text(test_doc)
pprint(chunks)

[
    '汽車法規是非題',
    '第2頁/共36',
    '36頁    \n題號',
    '題號 答案 題',
    '目  分類\n編號',
    '編號  \n001',
    '○  尊重生命是',
    '命是駕駛道德最重要的',
    '要的一點，我們開車時',
    '車時要處處顧 及行人',
    '行人，尤\n其應該注意',
    '注意讓老弱婦孺身心障',
    '心障礙者優先通行。',
    '。   10 \n00',
    '002  X  遵守',
    '遵守交通法規與秩序，',
    '序，只算是優良駕駛人',
    '駛人，與駕駛道德無關',
    '無關。   10',
    '003  ○',
    '汽油著火時，應用',
    '應用滅火器、泥沙或用',
    '或用水浸濕棉被、衣服',
    '衣服覆蓋撲滅。',
    '07 \n00'
]

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(separators=['○','X'],
                                               chunk_size=10,
                                               chunk_overlap=3)
chunks = text_splitter.split_text(test_doc)
pprint(chunks)

[
    '汽車法規是非題   \n第2頁/共36頁    \n題號 答案 題   目  分類\n編號  \n001  ',
    '○  尊重生命是駕駛道德最重要的一點，我們開車時要處處顧 及行人，尤\n其應該注意讓老弱婦孺身心障礙者優先通行。   
10 \n002  ',
    'X  遵守交通法規與秩序，只算是優良駕駛人，與駕駛道德無關。   10 \n003  ',
    '○  汽油著火時，應用滅火器、泥沙或用水浸濕棉被、衣服覆蓋撲滅。   07 \n00'
]

In [ ]:
from langchain_text_splitters import TokenTextSplitter
text_splitter = TokenTextSplitter(chunk_size=10,
                                  chunk_overlap=2)
chunks = text_splitter.split_text(test_doc)
pprint(chunks)

[
    '汽車法規是',
    '�是非題  ',
    '   \n第2�',
    '頁/共36頁',
    '��    \n題',
    '��號 答案',
    '�� 題   目',
    '��  分類\n',
    '�\n編號  \n',
    ' \n001  ○  尊',
    '��重生命是�',
    '駕駛道德',
    '德最重要的一',
    '的一點，我',
    '我們開車�',
    '�時要處�',
    '處顧 及�',
    '�行人，尤\n',
    '�\n其應該�',
    '�注意讓老',
    '老弱婦孺�',
    '�身心障�',
    '礙者優先�',
    '�通行。   10 ',
    ' 10 \n002  X  遵',
    '��守交通法',
    '法規與秩�',
    '�序，只�',
    '算是優良',
    '��駕駛人�',
    '人，與駕�',
    '�駛道德無',
    '無關。   10 \n',
    ' \n003  ○  汽',
    '��油著火',
    '火時，應',
    '��用滅火�',
    '��器、泥沙',
    '��或用水浸',
    '浸濕棉�',
    '被、衣服覆',
    '覆蓋撲�',
    '滅。   07 \n00'
]

In [ ]:
from langchain_text_splitters import TokenTextSplitter
text_splitter = TokenTextSplitter(model_name='gpt-4-turbo',
                                  chunk_size=10,
                                  chunk_overlap=2)
chunks = text_splitter.split_text(test_doc)
pprint(chunks)

[
    '汽車法規是非�',
    '非題   \n第2頁/共',
    '/共36頁    \n題號',
    '號 答案 題  ',
    '�   目  分類\n�',
    '\n編號  \n001  ○',
    ' ○  尊重生命是�',
    '是駕駛道德',
    '德最重要的一點，',
    '�，我們開車時',
    '�時要處處顧 �',
    '� 及行人，尤\n其',
    '\n其應該注意讓',
    '讓老弱婦孺',
    '孺身心障礙者�',
    '者優先通行。   ',
    '   10 \n002  X  �',
    ' 遵守交通法規',
    '規與秩序，只算',
    '只算是優良�',
    '駕駛人，與',
    '與駕駛道�',
    '道德無關。   10',
    ' 10 \n003  ○  �',
    ' 汽油著火時',
    '火時，應用滅火',
    '�火器、泥沙或用',
    '或用水浸濕棉',
    '棉被、衣服覆',
    '覆蓋撲滅。',
    '�。   07 \n00'
]

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(separators=[' \n'],
                                               chunk_size=10,
                                               chunk_overlap=2)
chunks = text_splitter.split_documents(docs)
pprint(chunks[15:20])

[
    Document(
        page_content=' \n題號 答案 題   目  分類\n編號 ',
        metadata={'source': 'https://ppt.cc/f9nc5x', 'page': 1}
    ),
    Document(
        page_content=' \n001  ○  尊重生命是駕駛道德最重要的一點，我們開車時要處處顧 
及行人，尤\n其應該注意讓老弱婦孺身心障礙者優先通行。   10',
        metadata={'source': 'https://ppt.cc/f9nc5x', 'page': 1}
    ),
    Document(
        page_content=' \n002  X  遵守交通法規與秩序，只算是優良駕駛人，與駕駛道德無關。   10',
        metadata={'source': 'https://ppt.cc/f9nc5x', 'page': 1}
    ),
    Document(
        page_content=' \n003  ○  汽油著火時，應用滅火器、泥沙或用水浸濕棉被、衣服覆蓋撲滅。   07',
        metadata={'source': 'https://ppt.cc/f9nc5x', 'page': 1}
    ),
    Document(
        page_content=' \n004  ○  禮讓與寬容是駕駛道德的最好表現。   10',
        metadata={'source': 'https://ppt.cc/f9nc5x', 'page': 1}
    )
]

## 7-2 Embedding 向量化

### RAG 第三步：文字轉向量

In [ ]:
embeddings_doc = [
    "天空是藍色的",
    "天空不是紅色的",
    "sky is blue",
    "莓果是藍色的",
    "我今天吃了漢堡"
]
embeddings = embeddings_model.embed_documents(embeddings_doc)
len(embeddings[0])

3072

In [ ]:
query = "天空的顏色是？"
embedded_query = embeddings_model.embed_query(query)

餘弦相似度

In [ ]:
import numpy as np
def cosine_similarity(a, b):
    return np.dot(a, b)

In [ ]:
for doc_res, doc in zip(embeddings, embeddings_doc):
    similarity = cosine_similarity(embedded_query,doc_res)
    print(f'"{doc}" 與問題的相似度：{similarity}')

"天空是藍色的" 與問題的相似度：0.7518601571046652
"天空不是紅色的" 與問題的相似度：0.6953254880809308
"sky is blue" 與問題的相似度：0.5886082651020517
"莓果是藍色的" 與問題的相似度：0.36777005752314007
"我今天吃了漢堡" 與問題的相似度：0.08466408965632069


### RAG 第四步：儲存到向量資料庫 Chroma

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from langchain_community.vectorstores import Chroma
Chroma.from_documents(documents=chunks,
                      embedding=embeddings_model,
                      persist_directory='/content/drive/MyDrive/db',
                      collection_metadata={"hnsw:space": "cosine"})

hnsw:space 有 l2、ip、cosine

In [ ]:
db = Chroma(persist_directory='/content/drive/MyDrive/db',
            embedding_function=embeddings_model)

In [ ]:
pprint(db.search('紅燈右轉', k=2, search_type='similarity'))

[
    Document(
        page_content=' \n235  ○  汽車行駛至交岔路口，欲右轉彎時，應距交岔路口 
30公尺前，顯示\n方向燈或手勢，換入外側車道、右轉車道或慢車道，行至路口後再行\n右轉。  02 ',
        metadata={'page': 12, 'source': 'https://ppt.cc/f9nc5x'}
    ),
    Document(
        page_content=' \n447  X  在交岔路口等候左轉之車輛，綠燈亮時即可搶先左轉。   02',
        metadata={'page': 24, 'source': 'https://ppt.cc/f9nc5x'}
    )
]

In [ ]:
pprint(db.max_marginal_relevance_search("紅燈右轉",k=2))

[
    Document(
        page_content=' \n235  ○  汽車行駛至交岔路口，欲右轉彎時，應距交岔路口 
30公尺前，顯示\n方向燈或手勢，換入外側車道、右轉車道或慢車道，行至路口後再行\n右轉。  02 ',
        metadata={'page': 12, 'source': 'https://ppt.cc/f9nc5x'}
    ),
    Document(
        page_content=' \n006  ○  綠燈允許你依序通過，但駕駛人仍應注意違規闖紅燈的人和車。   01',
        metadata={'page': 1, 'source': 'https://ppt.cc/f9nc5x'}
    )
]

In [ ]:
pprint(db.similarity_search_with_relevance_scores('紅燈右轉',k=2))

[
    (
        Document(
            page_content=' \n235  ○  汽車行駛至交岔路口，欲右轉彎時，應距交岔路口 
30公尺前，顯示\n方向燈或手勢，換入外側車道、右轉車道或慢車道，行至路口後再行\n右轉。  02 ',
            metadata={'page': 12, 'source': 'https://ppt.cc/f9nc5x'}
        ),
        0.6097202301025391
    ),
    (
        Document(
            page_content=' \n447  X  在交岔路口等候左轉之車輛，綠燈亮時即可搶先左轉。   02',
            metadata={'page': 24, 'source': 'https://ppt.cc/f9nc5x'}
        ),
        0.5948590040206909
    )
]

In [ ]:
embedded_query = embeddings_model.embed_query("紅燈右轉")
pprint(db.similarity_search_by_vector(embedded_query ,k=2))

[
    Document(
        page_content=' \n235  ○  汽車行駛至交岔路口，欲右轉彎時，應距交岔路口 
30公尺前，顯示\n方向燈或手勢，換入外側車道、右轉車道或慢車道，行至路口後再行\n右轉。  02 ',
        metadata={'page': 12, 'source': 'https://ppt.cc/f9nc5x'}
    ),
    Document(
        page_content=' \n447  X  在交岔路口等候左轉之車輛，綠燈亮時即可搶先左轉。   02',
        metadata={'page': 24, 'source': 'https://ppt.cc/f9nc5x'}
    )
]

## 7-3 檢索對話流程鏈

檢索器的search_type可以設定 similarity、mmr、similarity_score_threshold

In [ ]:
retriever = db.as_retriever(search_type="similarity",
                            search_kwargs={"k": 6})

In [ ]:
retrieved_docs = retriever.invoke("紅燈右轉")
print(f'傳回 {len(retrieved_docs)} 筆資料')

傳回 6 筆資料


In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

In [ ]:
str_parser = StrOutputParser()
template = (
    "請根據以下內容加上自身判斷回答問題:\n"
    "{context}\n"
    "問題: {question}"
    )
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | chat_model
    | str_parser
)

In [ ]:
print(chain.invoke("汽車駕駛人若喝酒後，會使反應遲延，視力變差。請問是否正確"))

根據提供的資訊，有兩個文件表示飲酒後會使視力能力變差和運動反射神經遲鈍，另一個文件則表示喝酒會使反應遲延和視力增加。根據多數文件的內容來看，汽車駕駛人若喝酒後，會使反應遲延，視力變差的說法是正確的。


In [ ]:
for chunk in chain.stream("汽車駕駛人若喝酒後，會使反應遲延，視力增加。請問是否正確"):
    print(chunk, end="", flush=True)

根據以上提供的內容，其中有一個文件指出飲酒後會使視覺能力變差，運動反射神經遲鈍，肇事率增加，而另一個文件則指出飲酒後會使反應遲延，視力增加。基於這兩個文件的內容，可以得知汽車駕駛人若喝酒後，會使反應遲延，視力增加這個說法是不正確的。因此，這個說法並不正確。

### 傳回關聯資料

In [ ]:
rag_chain_from_docs = (
    prompt
    | chat_model
    | StrOutputParser()
)

In [ ]:
from langchain_core.runnables import RunnableParallel

rag_chain_with_source = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

In [ ]:
def chat(query):
    output = {}
    curr_key = None
    for chunk in rag_chain_with_source.stream(query):
        for key in chunk:
            if key not in output:
                output[key] = chunk[key]
            else:
                output[key] += chunk[key]
            if key != curr_key:
                print(f"\n\n{key}: {chunk[key]}", end="", flush=True)
            else:
                print(chunk[key], end="", flush=True)
            curr_key = key
chat("汽車駕駛人若喝酒後，會使反應遲延，視力變差。是否正確")



question: 汽車駕駛人若喝酒後，會使反應遲延，視力變差。是否正確

context: [Document(page_content=' \n410  X  汽車駕駛人若喝酒後，會使反應遲延，視力增加。   08', metadata={'page': 22, 'source': 'https://ppt.cc/f9nc5x'}), Document(page_content=' \n409  ○  飲酒後，會使視覺能力變差，運動反射神經遲鈍，肇事率增加。   08', metadata={'page': 22, 'source': 'https://ppt.cc/f9nc5x'}), Document(page_content=' \n013  X  一個人飲酒後，雖然視力、聽覺以及判斷能力，都會變為遲鈍，但在\n其本人的感覺來說，卻很舒暢，所以酒後可以駕駛汽車。   08', metadata={'page': 1, 'source': 'https://ppt.cc/f9nc5x'}), Document(page_content=' \n398  ○  汽車行駛速度愈快，駕駛人的視野變窄、視力變差。   10', metadata={'page': 21, 'source': 'https://ppt.cc/f9nc5x'}), Document(page_content=' \n302  X  在高速行車狀況下，駕駛人視覺與平時一樣，沒有一點影響。   03', metadata={'page': 16, 'source': 'https://ppt.cc/f9nc5x'}), Document(page_content=' \n238  ○  駕駛人酒精濃度超過規定標準，駕車肇事致人重傷或死亡者，除處罰\n鍰外，吊銷其駕照，並不得再考領。但符合特定條件，且所受吊銷駕\n駛執照處分，執行已逾相關規定期間，並依規定完成酒駕防制教育或\n酒癮治療者，不在此限。   08', metadata={'page': 13, 'source': 'https://ppt.cc/f9nc5x'})]

answer: 是正確的。根據提供的文獻內容，喝酒後會使汽車駕駛人的反應遲延，視力變差，這些因素都會增加肇事的風險。因此，駕駛人喝酒後應該避免駕駛汽車，以確保自

In [ ]:
chat("紅燈可以右轉。是否正確")



question: 紅燈可以右轉。是否正確

context: [Document(page_content=' \n447  X  在交岔路口等候左轉之車輛，綠燈亮時即可搶先左轉。   02', metadata={'page': 24, 'source': 'https://ppt.cc/f9nc5x'}), Document(page_content=' \n200  ○  汽車在迴車前，應暫停並顯示左轉燈光或手勢，看清確無來往車輛，\n並注意行人通過，始得迴轉。   02', metadata={'page': 11, 'source': 'https://ppt.cc/f9nc5x'}), Document(page_content=' \n137  ○  汽車右轉彎時，應先顯示前後右邊方向燈，或由駕駛人作左臂向上手\n掌向右微曲之手勢。   02', metadata={'page': 7, 'source': 'https://ppt.cc/f9nc5x'}), Document(page_content=' \n235  ○  汽車行駛至交岔路口，欲右轉彎時，應距交岔路口 30公尺前，顯示\n方向燈或手勢，換入外側車道、右轉車道或慢車道，行至路口後再行\n右轉。  02 ', metadata={'page': 12, 'source': 'https://ppt.cc/f9nc5x'}), Document(page_content=' \n375  ○  汽車駕駛人行經有燈光號誌管制之交岔路口，紅燈右轉者，處新臺幣\n600～1,800元罰鍰。   02', metadata={'page': 20, 'source': 'https://ppt.cc/f9nc5x'}), Document(page_content=' \n452  X  行車欲超越前方車輛，應先打右方向燈。   04', metadata={'page': 24, 'source': 'https://ppt.cc/f9nc5x'})]

answer: 根據提供的資料，紅燈右轉者將會被處以罰鍰，因此紅燈時不可以右轉，這是不正確的。

### RAG 集大成：建立檢索對話代理

In [ ]:
from langchain.tools.retriever import create_retriever_tool

tool = create_retriever_tool(
    retriever=retriever,
    name="retriever_by_car_regulations",
    description="搜尋並返回汽車法規是非題內容",
)
tools = [tool]

In [ ]:
from langchain_core.prompts import MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages([
    ('system','你是一位善用工具的好助理, '
              '請自己判斷上下文來回答問題, 不要盲目地使用工具'),
    MessagesPlaceholder(variable_name="chat_history"),
    ('human','{input}'),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

In [ ]:
from langchain.agents import (
    AgentExecutor, create_openai_tools_agent)

agent = create_openai_tools_agent(chat_model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

In [ ]:
from langchain_community.chat_message_histories import (
    SQLChatMessageHistory)
from langchain_core.runnables.history import (
    RunnableWithMessageHistory)

memory = SQLChatMessageHistory(
        session_id="test_id",
        connection_string='sqlite:////content/drive/MyDrive/retriever.db'
    )

def window_messages(chain_input):
    if len(memory.messages) > 6:
        cur_messages = memory.messages
        memory.clear()
        for message in cur_messages[-6:]:
            memory.add_message(message)
    return

def add_history(agent_executor):
    agent_with_chat_history = RunnableWithMessageHistory(
        agent_executor,
        lambda session_id: memory,
        input_messages_key="input",
        history_messages_key="chat_history",
    )
    memory_chain = (
        RunnablePassthrough.assign(messages=window_messages)
        | agent_with_chat_history
    )
    return memory_chain

In [ ]:
memory_chain = add_history(agent_executor)

In [ ]:
while True:
    msg = input("我說：")
    if not msg.strip():
        break
    for chunk in memory_chain.stream(
        {"input": msg},
        config={"configurable": {"session_id": "test_id"}}):
        if 'output' in chunk:
            print(f"AI 回覆：{chunk['output']}", end="", flush=True)
    print('\n')

我說：汽車駕駛人若喝酒後，會使反應遲延，視力變差。是否正確
AI 回覆：根據汽車法規，飲酒後會使視覺能力變差，運動反射神經遲鈍，肇事率增加。因此，汽車駕駛人喝酒後的反應確實會變慢，視力也會變差，這是正確的說法。

我說：紅燈可以右轉嗎?
AI 回覆：根據汽車法規，紅燈時不可以右轉，違反紅燈右轉者將會面臨罰款。因此，紅燈時不可以進行右轉。

我說：左轉不應該禮讓直行車
AI 回覆：根據汽車法規，左轉車應該要遵守交通號誌，並且在有交通號誌規定的情況下，要給予直行車輛優先通行權。因此，左轉車應該要禮讓直行車輛。

我說：


## 7-4 總結文件的流程鏈

In [ ]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://blog.langchain.dev/nvidia-nim/")
langchain_docs = loader.load()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200,
                                               chunk_overlap=10)
langchain_splits = text_splitter.split_documents(langchain_docs)

### 不同的總結方式

stuff

In [ ]:
language_prompt = '使用繁體中文和台灣用詞'
prompt = ChatPromptTemplate.from_messages(
    [("system", "{language}總結以下內容：\n\n{text}")]
).partial(language=language_prompt)

In [ ]:
from langchain.chains.summarize import load_summarize_chain
chain = load_summarize_chain(llm=chat_model,
                             prompt=prompt,
                             chain_type="stuff")
print(chain.invoke(langchain_splits)['output_text'])

LangChain整合NVIDIA NIM，以優化GPU的LLM推論於RAG中。

NVIDIA NIM是一組易於使用的微服務，旨在加速企業中生成式人工智慧的部署。這個多功能運行時支援廣泛的AI模型，從開源社區模型到NVIDIA AI基礎模型，以及自定義AI模型。開發者可以利用行業標準的API快速建立企業級AI應用，只需幾行程式碼。NIM建立在堅固的基礎上，包括像NVIDIA Triton推理伺服器、NVIDIA TensorRT、NVIDIA TensorRT-LLM和PyTorch等推理引擎，旨在促進規模化無縫AI推論，確保您能夠信心滿滿地在任何地方部署AI應用。對於那些轉向將LLM應用從原型化轉為生產環境的企業，LangChain對整合NVIDIA NIM感到非常興奮。

LangChain新增了支援NIM的整合套件，以開始整合，您需要安裝我們專用的整合套件：pip install langchain_nvidia_ai_endpoints。接著，您可以匯入模型，例如：from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings, ChatNVIDIA。

LangChain整合了NVIDIA NIM，以支援GPU優化的LLM推論於RAG中，進一步提升企業AI應用的效率和性能。


MapReduce

In [ ]:
reduce_prompt = ChatPromptTemplate.from_messages(
    [("system", "{language}, 以下是文件內容：\n"
                "{text}\n"
                "將這些內容進行總結且保持核心內容")]
).partial(language=language_prompt)

map_prompt = ChatPromptTemplate.from_messages(
    [("system", "{language}, 以下是一組文件串列：\n"
                "{text}\n"
                "根據此文件串列, 請作摘要並確保核心內容")]
).partial(language=language_prompt)

In [ ]:
chain = load_summarize_chain(llm=chat_model,
                             combine_prompt=reduce_prompt,
                             map_prompt=map_prompt,
                             chain_type="map_reduce")
print(chain.invoke(langchain_splits)['output_text'])

，這份文件串列主要介紹了LangChain如何整合NVIDIA NIM進行GPU優化的LLM推論，以提高推論效率和速度。同時也包含了LangSmith文件的應用和高級檢索方法HyDE，以及相關的套件安裝和模組功能。此外，文件還涵蓋了環境永續發展、農村發展計畫、合作消息等主題，強調永續發展的重要性並提出解決方案和合作計畫。整體而言，這份文件串列提供了豐富的資訊，幫助讀者了解如何應用NVIDIA NIM和LangChain進行GPU優化的LLM推論，並探索相關的應用和技術挑戰，同時也呼應了環境永續發展的重要性和合作共識。


In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [("system", "{language}, 以下是文件的開頭內容：\n"
                "{text}\n"
                "將這些內容進行總結且保持核心內容")]
).partial(language=language_prompt)

refine_prompt = ChatPromptTemplate.from_messages(
    [("system", "{language}, 你的工作是撰寫綜合摘要\n"
                "這是目前的摘要成果：{existing_answer}\n"
                "藉由底下的額外內容"
                "（若需要的話）請再補強摘要內容：\n"
                "------------\n"
                "{text}\n"
                "------------\n"
                "如果這些額外內容沒有用，請返回原始摘要。")
    ]
).partial(language=language_prompt)

In [ ]:
chain = load_summarize_chain(chat_model,
                             question_prompt=prompt,
                             refine_prompt=refine_prompt,
                             chain_type="refine")
print(chain.invoke(langchain_splits)['output_text'])

在這段程式碼示範中，我們展示了如何使用 prompt、model 和 StrOutputParser 來建立一個最終的鏈結，並結合假設的文件檢索器來回答問題。透過這個最終鏈結，我們可以更有效地處理問題並提供答案。此外，Rakuten Group 利用 LangChain 和 LangSmith 打造出優質產品，為其商業客戶和員工提供服務。程式碼包括了對於問題的文件檢索以及對回答鏈結的串流處理，進一步強化了程式的功能和應用價值。在程式中，我們使用 yield 回傳 tokens 以便串流最終鏈結，讓我們一起來試試看：for s in final_chain.stream("how can langsmith help with testing"): print(s, end="")。這段程式碼充分展示了如何透過串流處理來處理最終鏈結，提升了程式的效率和實用性。進一步，這個程式也提供了Tags By LangChain，並鼓勵用戶訂閱通訊以獲取LangChain團隊和社群的更新。在訂閱過程中，系統會處理您的申請，若成功您將收到確認訂閱的連結，若有任何錯誤發生，請再試一次。這些額外內容進一步強化了程式的全面性和用途，讓整體功能更加完善。此外，還有有關Reflection Agents、Plan-and-Execute Agents和LangChain釋出註記的內容，這些內容也補充了程式的資訊，讓使用者更深入了解LangChain的相關內容。LangChain 在 2/5 週釋出了新版本，並與 CommandBar 合作推出 Copilot 使用者助手，這些新訊息進一步擴展了 LangChain 的應用範疇。


## 7-5 其他的分割器

In [ ]:
json_example={
    "school": {
        "name": "精英中學",
        "address": {
            "street": "高科技大道200號",
            "city": "台北",
            "zip_code": "100"
        },
        "departments": [
            {
                "name": "數學系",
                "courses": [
                    {
                        "course_name": "高等數學",
                        "teacher": "李老師",
                        "students": ["張三", "李四", "王五"]
                    },
                    {
                        "course_name": "線性代數",
                        "teacher": "周老師",
                        "students": ["趙六", "孫七", "周八"]
                    }
                ]
            },
            {
                "name": "物理系",
                "courses": [
                    {
                        "course_name": "量子物理",
                        "teacher": "張老師",
                        "students": ["陳九", "鄭十"]
                    }
                ]
            }
        ]
    }
}

In [ ]:
from langchain.text_splitter import RecursiveJsonSplitter
splitter = RecursiveJsonSplitter(max_chunk_size=30)
docs = splitter.create_documents(texts=[json_example],
                                 convert_lists=True)
pprint(docs[:5])

[
    Document(
        page_content='{"school": {"name": "\\u7cbe\\u82f1\\u4e2d\\u5b78", "address": {"street": 
"\\u9ad8\\u79d1\\u6280\\u5927\\u9053200\\u865f"}}}'
    ),
    Document(page_content='{"school": {"address": {"city": "\\u53f0\\u5317", "zip_code": "100"}}}'),
    Document(page_content='{"school": {"departments": {"0": {"name": "\\u6578\\u5b78\\u7cfb"}}}}'),
    Document(
        page_content='{"school": {"departments": {"0": {"courses": {"0": {"course_name": 
"\\u9ad8\\u7b49\\u6578\\u5b78"}}}}}}'
    ),
    Document(
        page_content='{"school": {"departments": {"0": {"courses": {"0": {"teacher": "\\u674e\\u8001\\u5e2b"}}}}}}'
    )
]

### markdown

In [ ]:
md = '''
# 時間管理的藝術

時間管理是一項關鍵技能，可以幫助個人有效地利用時間，提高生產力和效率。
本文件旨在提供一些基本的時間管理技巧，幫助讀者更好地規劃和利用自己的時間。

## 為什麼時間管理如此重要？

在快節奏的現代生活中，時間成為了一種寶貴的資源。
良好的時間管理不僅可以幫助我們完成更多的工作，還可以提高生活質量，
給予我們更多時間去追求個人興趣和與家人、朋友相處的時光。

## 基本時間管理技巧

### 設定目標

- **確定優先順序**：了解哪些任務最重要，哪些可以稍後處理。
- **SMART目標**：設定具體（Specific）、可衡量（Measurable）、
可達成（Achievable）、相關（Relevant）、時間限定（Time-bound）的目標。

### 規劃你的時間

- **每日計劃**：每天制定一個實際可行的待辦事項清單。
- **時間塊劃分**：將一天分成幾個時間塊，每個時間塊分配特定的任務。

### 避免拖延

- **使用番茄工作法**：專注工作25分鐘，然後休息5分鐘。
- **設定獎勵**：完成任務後給自己一些小獎勵。

## 工具和應用

- **Google Calendar**：用於時間規劃和會議安排。
- **Trello**：一個項目管理工具，有助於跟蹤任務和進度。
- **Pomodoro Timer**：一個簡單的線上番茄鐘工具。

## 結語

有效的時間管理要求持之以恆的努力和自我反思。透過實踐上述技巧，
您將能夠更有效地利用您的時間，達成個人和專業目標，同時享有更豐富的個人生活。
'''

In [ ]:
from langchain_text_splitters import MarkdownHeaderTextSplitter

In [ ]:
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]

markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on, strip_headers=True)

md_header_splits = markdown_splitter.split_text(md)
pprint(md_header_splits)

[
    Document(
        page_content='時間管理是一項關鍵技能，可以幫助個人有效地利用時間，提高生產力和效率。\n本文件旨在提供一些基
本的時間管理技巧，幫助讀者更好地規劃和利用自己的時間。',
        metadata={'Header 1': '時間管理的藝術'}
    ),
    Document(
        page_content='在快節奏的現代生活中，時間成為了一種寶貴的資源。\n良好的時間管理不僅可以幫助我們完成更多的工
作，還可以提高生活質量，\n給予我們更多時間去追求個人興趣和與家人、朋友相處的時光。',
        metadata={'Header 1': '時間管理的藝術', 'Header 2': '為什麼時間管理如此重要？'}
    ),
    Document(
        page_content='- **確定優先順序**：了解哪些任務最重要，哪些可以稍後處理。\n- 
**SMART目標**：設定具體（Specific）、可衡量（Measurable）、\n可達成（Achievable）、相關（Relevant）、時間限定（Time
-bound）的目標。',
        metadata={'Header 1': '時間管理的藝術', 'Header 2': '基本時間管理技巧', 'Header 3': '設定目標'}
    ),
    Document(
        page_content='- **每日計劃**：每天制定一個實際可行的待辦事項清單。\n- 
**時間塊劃分**：將一天分成幾個時間塊，每個時間塊分配特定的任務。',
        metadata={'Header 1': '時間管理的藝術', 'Header 2': '基本時間管理技巧', 'Header 3': '規劃你的時間'}
    ),
    Document(
        page_content='- **使用番茄工作法**：專注工作25分鐘，然後休息5分鐘。\n- 
**設定獎勵**：完成任務後給自己一些小獎勵。',
        metadata={'Header 1': '時間管理的藝術', 'Header 2': '基本時間管理技巧', 'Header 3': '避免拖延'}
    ),
    Document(
        page_content='- **Google Calendar**：用於時間規劃和會議安排。\n- 
**Trello**：一個項目管理工具，有助於跟蹤任務和進度。\n- **Pomodoro Timer**：一個簡單的線上番茄鐘工具。',
        metadata={'Header 1': '時間管理的藝術', 'Header 2': '工具和應用'}
    ),
    Document(
        page_content='有效的時間管理要求持之以恆的努力和自我反思。透過實踐上述技巧，\n您將能夠更有效地利用您的時間
，達成個人和專業目標，同時享有更豐富的個人生活。',
        metadata={'Header 1': '時間管理的藝術', 'Header 2': '結語'}
    )
]

In [ ]:
chunk_size = 50
chunk_overlap = 10

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap
)

splits = text_splitter.split_documents(md_header_splits)
pprint(splits[:2])

[
    Document(
        page_content='時間管理是一項關鍵技能，可以幫助個人有效地利用時間，提高生產力和效率。',
        metadata={'Header 1': '時間管理的藝術'}
    ),
    Document(
        page_content='本文件旨在提供一些基本的時間管理技巧，幫助讀者更好地規劃和利用自己的時間。',
        metadata={'Header 1': '時間管理的藝術'}
    )
]

### html

In [ ]:
from langchain_text_splitters import HTMLHeaderTextSplitter

In [ ]:
url = "https://zh.wikipedia.org/zh-tw/%E8%94%A1%E8%8B%B1%E6%96%87"

headers_to_split_on = [
    ("h1", "Header 1"),
    ("h2", "Header 2"),
    ("h3", "Header 3"),
    ("h4", "Header 4"),
]

html_splitter = HTMLHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on
)

html_header_splits = html_splitter.split_text_from_url(url)
pprint(html_header_splits[5:8])

[
    Document(
        page_content='1986年，由於自身的國際貿易專業知識，蔡英文首次加入中華民國政府擔任相關官員[40]，在1980年代後
期開始參與臺灣對外的經濟貿易談判[38]。她後來成為首席談判代表，在第一線參與關鍵談判[38][40]。同時還加入政府，開始出
任一系列重要、且政策導向的職務[71]。在1993年至2000年，她擔任經濟部貿易調查委員會委員[82]。在1995年至1998年，她獲委
任行政院公平交易委員會委員[61]、及內政部著作權委員會委員[69][84]。  
\n在1990年到2002年期間，臺灣的經濟體系試圖加入關稅暨貿易總協定、世界貿易組織[38]。從1992年至2000年，她加入中華民國
經濟部，擔任國際經濟組織的首席法律顧問[43][61]，協助臺灣申請加入世界貿易組織[69][85]。從1998年起，她更直接參與臺灣
與世界貿易組織的談判，並向政府提出建議[86]，從而發揮重要作用[87][88][89]。  
\n另外在1990年代，她還經常性參與亞太經濟合作會議的會議，曾擔任貿易及投資委員會服務業小組主席[38]。2000年，在給民主
進步黨政黨領袖陳水扁上過「世界貿易組織專案課程」後，讓陳水扁對她讚賞不已，後來因看重其主導「特殊的國與國關係」、多
次參與國際談判、及參與海峽兩岸「辜汪會談」等，決定延攬當時尚不知名的她進入執政團隊[33][78]。',
        metadata={'Header 1': '蔡英文', 'Header 2': '早期工作', 'Header 3': '貿易談判代表'}
    ),
    Document(
        page_content='1994年，在經過長期的國際事務歷練後，蔡英文深深獲得時任中華民國總統李登輝等政府高級官員的賞識[
72]，並欽點她擔任李登輝政府的兩岸問題專家[50][69][86]。從1994年至1998年，蔡英文擔任行政院大陸委員會諮詢委員會委員[6
1][69]，協助統籌處理海峽兩岸關係事務[90][84]。其政治生涯因這次機緣，長期與兩岸問題有密切關係[90]。在1994年至1995年
，她擔任《香港澳門關係條例》起草研究小組召集人，負責起草管理臺灣、香港與澳門間經濟關係的法案[69][91]。  
\n從1990年代後期起，海峽兩岸關係歷經數次波動[38]。1998年，她以行政院大陸委員會諮詢委員身分，出席第二次辜汪會談，擔
任隨團發言人[80]。1999年至2000年，李登輝聘請她兼任中華民國總統府國家安全會議諮詢委員[61]，及出任第六屆國家統一委員
會研究委員，成為身邊的重要幕僚之一[33][84]。1999年，在國家安全會議的專案委託下[92]，擔任李登輝執政最著名的「特殊的
國與國關係」研究小組召集人[43][77][84]。  
\n在這期間，她由中華民國國家安全局丁渝洲提撥研究經費[93][94]，參與「特殊的國與國關係」解決方案的起草[77][95]，是研
究報告的主筆人[90][72][92]，此後李登輝採用該研究成果，公開提出「特殊的國與國關係」理論[92]，導致1999年海峽兩岸關係
陷入緊張[50]。這段歷史曾讓馬英九在出席競選活動時要求說明[90]，中國共產黨也藉此指控她是「分裂中國的始作俑者」[92]。'
,
        metadata={'Header 1': '蔡英文', 'Header 2': '早期工作', 'Header 3': '兩岸關係幕僚'}
    ),
    Document(
        page_content='2000年，民主進步黨候選人陳水扁當選總統，並在5月20日執政[96]。這是該黨首次執政[78]，對內需要擴
展民意支持、鞏固政權基礎，對外則要穩定海峽兩岸關係、安撫美國[33]。經過邀請，陳水扁任命蔡英文擔任行政院大陸委員會主
任委員[43][61][96]，並參加國家安全會議，開始其高層政治生涯[30]。這是她第一次真正步入政壇[72]，也正式與中華人民共和
國交手[92][97][98]。作為陳水扁任內首位行政院大陸委員會主任委員，她還是行政院大陸委員會首位女性主任委員[72]、及首位
完成4年任期的主任委員[92]。  
\n在這期間，她認為「特殊的國與國關係」是開創海峽兩岸關係新架構的提議[35]，並緊急阻止陳水扁為了改善兩岸關係而接受「
九二共識」的做法[92]。在這期間，蔡英文最重要的海峽兩岸政策，是在第一線與中國大陸協商，最後促成「小三通」模式自2001
年1月1日起開放，實現小規模的通商、通航和通郵[90][99]。她還曾自稱是臺灣人，也是文化和血緣上的中國人[100]。2001年，陳
水扁對中國大陸釋出善意，又提出「統合論」，蔡英文則避開新聞媒體不斷追問的實質內涵，只以「善意」和「彈性」解釋[33]。
蔡英文也以其英語口才和精確表達聞名，在2002年擔任調解人員，前往美國解釋陳水扁提出的臺灣和中國是「一邊一國」的說法[77
]。2004年，她還參與公民投票辯論，不過認為這具有表演性質[33]。  
\n在缺乏和中華人民共和國交流和合作的管道下[101]，行政院大陸委員會還協調立法院通過《臺灣地區與大陸地區人民關係條例》
修法[102]、兩岸春節包機直航[103]、完成臺灣和香港航權談判等[104]。蔡英文致力處理、主持海峽兩岸政策的發展[51][38][98]
，也因為這是一份備受矚目的困難任務，讓她在臺灣很快受到注意[46]。她在連續4年的內閣閣員滿意度輿論調查都是第一名[92]。
相對地，中華人民共和國政府則是把其視為「鬥爭與談判的對手」，並運用「文攻武嚇」的策略[92]。',
        metadata={'Header 1': '蔡英文', 'Header 2': '步入政壇', 'Header 3': '陸委會主委'}
    )
]

In [ ]:
chunk_size = 500
chunk_overlap = 30
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap
)

splits = text_splitter.split_documents(html_header_splits)
pprint(splits[5:8])

[
    Document(
        page_content='編輯連結  \n條目討論  \n臺灣正體  
\n不转换简体繁體大陆简体香港繁體澳門繁體大马简体新加坡简体臺灣正體  \n閱讀檢視原始碼檢視歷史  \n工具  \n移至側邊欄 
隱藏  \n工具  \n操作  \n閱讀檢視原始碼檢視歷史  \n一般  
\n連結至此的頁面相關變更上傳檔案特殊頁面固定連結頁面資訊引用此頁面取得短網址下載QR碼維基數據項目  \n列印/匯出  
\n下載為PDF可列印版  \n其他專案  \n維基共享資源維基新聞維基語錄維基文庫  \n維基百科，自由的百科全書',
        metadata={'Header 1': '蔡英文'}
    ),
    Document(
        page_content='蔡英文\xa0中華民國第14－15任總統選舉：2016、2020就任日期2016年5月20日行政院院長林全（2016－20
17）賴清德（2017－2019）蘇貞昌（2019－2023）陳建仁（2023－2024）副總統陳建仁（第14任）賴清德（第15任）秘書長前任馬
英九繼任賴清德（候任）多數票',
        metadata={'Header 1': '蔡英文'}
    ),
    Document(
        page_content='第7任國家安全會議主席就任日期2016年5月20日副職陳建仁（2016－2020）賴清德（2020－2024）秘書長
吳釗燮（2016－2017）嚴德發（2017－2018）李大維（2018－2020）顧立雄（2020－2024）前任馬英九繼任賴清德\xa0民主進步黨
第13、15、17任主席任期2020年5月20日—2022年11月30日辭職秘書長林錫耀前任卓榮泰繼任陳其邁（代理）賴清德（正任）任期201
4年5月28日—2018年11月28日辭職秘書長吳釗燮洪耀福前任蘇貞昌繼任林右昌（代理）卓榮泰（正任）任期2008年5月20日—2012年1
月14日辭職秘書長王拓吳乃仁蘇嘉全前任謝長廷（代理）陳水扁（正任）繼任陳菊（代理）蘇貞昌（正任）\xa0中華民國第26任行
政院副院長任期2006年1月25日—2007年5月21日總統陳水扁行政院院長蘇貞昌前任吳榮義繼任邱義仁\xa0中華民國第10任行政院消費
者保護會主任委員任期2006年1月25日—2007年5月21日行政院院長蘇貞昌前任吳榮義繼任邱義仁\xa0中華民國第6屆立法委員任期200
5年2月1日—2006年1月23日繼任吳明敏選區全國不分區\xa0中華民國第6任行政院',
        metadata={'Header 1': '蔡英文'}
    )
]

### code

In [ ]:
from langchain_text_splitters import Language

In [ ]:
[e.value for e in Language]

['cpp',
 'go',
 'java',
 'kotlin',
 'js',
 'ts',
 'php',
 'proto',
 'python',
 'rst',
 'ruby',
 'rust',
 'scala',
 'swift',
 'markdown',
 'latex',
 'html',
 'sol',
 'csharp',
 'cobol',
 'c',
 'lua',
 'perl']

In [ ]:
RecursiveCharacterTextSplitter.get_separators_for_language(
    Language.PYTHON)

['\nclass ', '\ndef ', '\n\tdef ', '\n\n', '\n', ' ', '']

In [ ]:
python_code = """
def hello_world():
    print("Hello, World!")

# 呼叫函式
hello_world()
"""
python_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON, chunk_size=50, chunk_overlap=0
)
python_docs = python_splitter.create_documents([python_code])
for doc in python_docs:
    print(doc.page_content)
    print('.'*10)

def hello_world():
    print("Hello, World!")
..........
# 呼叫函式
hello_world()
..........


In [ ]:
csharp_code = """
using System;
class Program
{
    static void Main()
    {
        int age = 30; // 根據需要更改年齡值

        // 根據年齡對人進行分類，並輸出到控制台
        if (age < 18)
        {
            Console.WriteLine("年齡小於18：未成年");
        }
        else if (age >= 18 && age < 65)
        {
            Console.WriteLine("年齡在18至64之間：成年人");
        }
        else
        {
            Console.WriteLine("年齡65歲或以上：老年人");
        }
    }
}

"""

In [ ]:
RecursiveCharacterTextSplitter.get_separators_for_language(
    language=Language.CSHARP)

['\ninterface ',
 '\nenum ',
 '\nimplements ',
 '\ndelegate ',
 '\nevent ',
 '\nclass ',
 '\nabstract ',
 '\npublic ',
 '\nprotected ',
 '\nprivate ',
 '\nstatic ',
 '\nreturn ',
 '\nif ',
 '\ncontinue ',
 '\nfor ',
 '\nforeach ',
 '\nwhile ',
 '\nswitch ',
 '\nbreak ',
 '\ncase ',
 '\nelse ',
 '\ntry ',
 '\nthrow ',
 '\nfinally ',
 '\ncatch ',
 '\n\n',
 '\n',
 ' ',
 '']

In [ ]:
csharp_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.CSHARP, chunk_size=50, chunk_overlap=0
)
csharp_docs = csharp_splitter.create_documents([csharp_code])
for doc in csharp_docs:
    print(doc.page_content)
    print('.'*10)

using System;
..........
class Program
{
    static void Main()
    {
..........
int age = 30; // 根據需要更改年齡值
..........
// 根據年齡對人進行分類，並輸出到控制台
..........
if (age < 18)
        {
..........
Console.WriteLine("年齡小於18：未成年");
..........
}
        else if (age >= 18 && age < 65)
..........
{
..........
Console.WriteLine("年齡在18至64之間：成年人");
..........
}
        else
        {
..........
Console.WriteLine("年齡65歲或以上：老年人");
..........
}
    }
}
..........
